In [1]:
import copy
import pandas as pd
from nasbench import api
from tqdm.notebook import tqdm

nasbench = api.NASBench("nasbench_full.tfrecord")




Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 124 seconds


In [ ]:
INPUT = "input"
OUTPUT = "output"
CONV3X3 = "conv3x3-bn-relu"
CONV1X1 = "conv1x1-bn-relu"
MAXPOOL3X3 = "maxpool3x3"

NUM_VERTICES = 7
MAX_EDGES = 9

EDGE_SPOTS = NUM_VERTICES * (NUM_VERTICES - 1) / 2  # Upper triangular matrix
OP_SPOTS = NUM_VERTICES - 2  # Input/output vertices are fixed

ALLOWED_OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]
ALLOWED_EDGES = [0, 1]  # Binary adjacency matrix"

In [29]:
# query max accuracy model from dataset
all_data = []
for spec_hash in tqdm(nasbench.hash_iterator()):
    data = []
    results = nasbench.get_metrics_from_hash(spec_hash)
    results = copy.deepcopy(results)

    acc = results[1].get(108)
    training_time = sum(map(lambda x: x['final_training_time'], acc))/3
    train_accuracy = sum(map(lambda x: x['final_train_accuracy'], acc))/3
    validation_accuracy = sum(map(lambda x: x['final_validation_accuracy'], acc))/3
    test_accuracy = sum(map(lambda x: x['final_test_accuracy'], acc))/3

    results = [
        spec_hash,
        results[0]["trainable_parameters"],
        training_time,
        train_accuracy,
        validation_accuracy,
        test_accuracy,
    ]
    all_data.append(results)

  0%|          | 0/423624 [00:00<?, ?it/s]

In [30]:
df = pd.DataFrame(
    all_data,
    columns=[
        "Hash",
        "Trainable Parameters",
        "Training Time",
        "Train Accuracy",
        "Validation Accuracy",
        "Test Accuracy",
    ],
)

In [31]:
df._get_numeric_data().min()

Trainable Parameters    227274.000000
Training Time              284.500336
Train Accuracy               0.102197
Validation Accuracy          0.094752
Test Accuracy                0.099760
dtype: float64

In [32]:
df._get_numeric_data().max()

Trainable Parameters    4.997927e+07
Training Time           5.521803e+03
Train Accuracy          1.000000e+00
Validation Accuracy     9.505542e-01
Test Accuracy           9.431758e-01
dtype: float64

In [33]:
df._get_numeric_data().median()

Trainable Parameters    5.356682e+06
Training Time           1.560519e+03
Train Accuracy          9.999666e-01
Validation Accuracy     9.123264e-01
Test Accuracy           9.063836e-01
dtype: float64

In [34]:
df._get_numeric_data().mean()

Trainable Parameters    8.459762e+06
Training Time           1.932403e+03
Train Accuracy          9.917610e-01
Validation Accuracy     9.024339e-01
Test Accuracy           8.967985e-01
dtype: float64

In [35]:
df._get_numeric_data().std()

Trainable Parameters    8.558503e+06
Training Time           9.187951e+02
Train Accuracy          5.965972e-02
Validation Accuracy     5.864747e-02
Test Accuracy           5.799575e-02
dtype: float64